In [17]:
import subprocess
import tempfile
import os
import pandas as pd
import time
import glob

In [23]:
def run_cpp_code_with_file_input(code: str, input_file_path: str) -> str:
    # Create a temporary directory to hold the C++ file and executable
    with tempfile.TemporaryDirectory() as temp_dir:
        cpp_file_path = os.path.join(temp_dir, "code.cpp")
        executable_path = os.path.join(temp_dir, "code")
        
        # Write the C++ code to a file
        with open(cpp_file_path, "w") as cpp_file:
            cpp_file.write(code)
        
        # Compile the C++ code
        compile_process = subprocess.run(["g++", cpp_file_path, "-o", executable_path], capture_output=True, text=True)
        if compile_process.returncode != 0:
            # Compilation failed
            return f"Compilation Failed:\n{compile_process.stderr}"
        
        # Run the compiled executable with input redirected from the input file
        start_time = time.time()
        with open(input_file_path, 'r') as input_file:
            run_process = subprocess.run(executable_path, stdin=input_file, capture_output=True, text=True, universal_newlines=True)
            if run_process.returncode != 0:
                # Runtime error
                return f"Runtime Error:\n{run_process.stderr}"
        end_time = time.time()
        return run_process.stdout, (end_time - start_time)


def eval_output(output: str, expected_output_file: str) -> bool:
    with open(expected_output_file, 'r') as expected_file:
        expected_output = expected_file.read()
        return output.strip() == expected_output.strip()
    

def run_tcs(code: str, sample_output_folder: str, hidden_output_folder) -> bool:
    for folder in [sample_output_folder, hidden_output_folder]:
        input_files = glob.glob(os.path.join(folder, "input.*.txt"))
        for input_file in input_files:
            # print('processing:', input_file)
            expected_output_file = input_file.replace('input', 'output')  # Assuming expected output file naming convention
            actual_output, time = run_cpp_code_with_file_input(code, input_file)
            if actual_output is None or not eval_output(actual_output, expected_output_file):
                print('Failed on test case', input_file)
                return False
    
    return True 

In [3]:
df = pd.read_csv("../datasets/improvement_pairs_additional_metadata.csv", sep="\t")

In [4]:
# Assuming 'input.txt' is a file in the current directory containing the input for the C++ program
input_file_path = '../datasets/codenet/public_test_cases/p00849/input.0.txt'
sample_code = df.at[3, 'code_v1']

In [15]:
output, tot_time = run_cpp_code_with_file_input(sample_code, input_file_path)
print(f"total time: {tot_time:.2f} seconds")
print(eval_output(output, '../datasets/codenet/public_test_cases/p00849/output.0.txt'))

total time: 0.46 seconds
True


In [16]:
print(df.head())

      user_id problem_id language submission_id_v0 submission_id_v1  \
0  u116766943     p00849      C++       s624998539       s546809015   
1  u116766943     p00849      C++       s546809015       s829771916   
2  u116766943     p00849      C++       s812631681       s154463767   
3  u116766943     p00849      C++       s363969251       s061168313   
4  u116766943     p00849      C++       s061168313       s755775167   

   cpu_time_v0  cpu_time_v1  memory_v0  memory_v1            status_v0  \
0       9370.0       9250.0     1232.0     1228.0  Time Limit Exceeded   
1       9250.0       8780.0     1228.0     1228.0  Time Limit Exceeded   
2       8700.0       8030.0     1232.0     1232.0  Time Limit Exceeded   
3       8500.0        920.0     1252.0    12164.0  Time Limit Exceeded   
4        920.0        420.0    12164.0     9780.0             Accepted   

             status_v1  improvement_frac  \
0  Time Limit Exceeded              1.28   
1  Time Limit Exceeded              5.08

In [24]:
print(run_tcs(sample_code, "../datasets/codenet/public_test_cases/p00849/", ""))

KeyboardInterrupt: 